In [ ]:
# подключение к LLama2, взято из https://www.kaggle.com/code/swarnimsawane/test-llm

!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512,
    n_gpu_layers=32
    )

lcpp_llm.params.n_gpu_layers

In [ ]:
# парсинг ответов txt в tsv Vicuna
# прогона от 18.04.24 сценария Label, category, explanation, correction

from google.colab import drive
import csv

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/data/vicuna_cola_eng_18.04.24_LabelCategoryExplanationCorrection1.txt') as f:
  df = f.readlines()
  y_pred = []
  categ = []
  expl = []
  k = []
  c = []
  corr = []
  for i in df:
    i = i.split('2.')
    k.append(i[0])
    l = i[1].split('3.')
    c.append(l[0])
    e = l[1].split('4.')
    expl.append(e[0])
    corr.append(e[-1])
for i in k:
    if '1. Yes' in i or 'here are two' in i or 'here are several' in i or 'here are a few' in i or 'here is one' in i or 'your sentence contains a' in i or 'here is a' in i:
          b = '0'
          y_pred.append(b)
    else:
      # print(n)
      b = '1'
      y_pred.append(b)
for i in c:
  if 'orpholog' in i:
    cat = 'morphology'
  if 'ynta' in i:
    cat = 'syntax'
  if 'emantic' in i:
    cat = 'semantics'
  if 'ynta' in i and 'emantic' in i:
    cat = 'syntax, semantics'
  if 'ynta' in i and 'orpholog' in i:
    cat = 'syntax, morphology'
  if 'emantic' in i and 'orpholog' in i:
    cat = 'semantics, morphology'
  if 'emantic' not in i and 'orpholog' not in i and 'ynta' not in i:
    cat = 'none'
  categ.append(cat)
print(y_pred)

with open('/content/drive/MyDrive/feeding/data/tsv/vicuna_cola_eng_18.04.24_LabelCategoryExplanationCorrection1.tsv', 'a') as f:
  col = ['label', 'category', 'explanation', 'correction']
  writer = csv.writer(f, delimiter='\t')
  writer.writerow(['label', 'category', 'explanation', 'correction'])
  for label, category, explanation, correction in zip(y_pred, categ, expl, corr):
      writer.writerow([label, category, explanation, correction])

In [ ]:
# парсинг ответов txt в tsv Llama2
# прогона от 13.05.24 сценария Label, category, explanation, correction

from google.colab import drive
import csv

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/data/llama_cola_eng_13.05.24_LabelCategoryExplanationCorrection1.txt') as f:
  df = f.read()
  y_pred = []
  categ = []
  k = []
  expl = []
  cat = []
  corr = []
  df = df.split('''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
    USER: I wrote a sentence in En''')[1:]
  for i in df:
    i = i.split('ASSISTANT:')[1].split('2.')
    k.append(i[0])
    l = i[1].split('3.')
    cat.append(l[0])
    expl.append(l[1].split('4.')[0])
    corr.append(l[1].split('4.')[1].split('Llama.generate: prefix-match hit')[0])
for i in k:
    if '1. Yes' in i or 'here are two' in i or 'here are several' in i or 'here are a few' in i or 'here is one' in i or 'your sentence contains a' in i or 'here is a' in i and 'No' not in i:
          b = '0'
          y_pred.append(b)
    elif '1. No' in i:
      # print(n)
      b = '1'
      y_pred.append(b)
for i in cat:
  if 'orpholog' in i:
    cat = 'morphology'
  if 'ynta' in i:
    cat = 'syntax'
  if 'emantic' in i:
    cat = 'semantics'
  if 'ynta' in i and 'emantic' in i:
    cat = 'syntax, semantics'
  if 'ynta' in i and 'orpholog' in i:
    cat = 'syntax, morphology'
  if 'emantic' in i and 'orpholog' in i:
    cat = 'semantics, morphology'
  if 'emantic' not in i and 'orpholog' not in i and 'ynta' not in i:
    cat = 'none'
  categ.append(cat)
print(y_pred)
print(len(y_pred))
print(len(corr))
print(len(expl))

with open('/content/drive/MyDrive/feeding/data/tsv/llama_cola_eng_13.05.24_LabelCategoryExplanationCorrection1.tsv', 'a') as f:
  col = ['label', 'category', 'explanation', 'correction']
  writer = csv.writer(f, delimiter='\t')
  writer.writerow(['label', 'category', 'explanation', 'correction'])
  for label, category, explanation, correction in zip(y_pred, categ, expl, corr):
      writer.writerow([label, category, explanation, correction])

In [ ]:
# label 1

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    prompt = f"Are there any strict grammar mistakes in my sentence '{t}'? Answer like 'Yes' or 'No'"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label 2

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    prompt = f"Are there any strict grammar mistakes in my sentence '{t}'? Answer like 'Yes,' or 'No,'. Start your respose with 'Yes' or 'No'"
    prompt_template=f'''SYSTEM: You are an English teacher. Always answer as helpfully, with good explanations of grammar and lexical mistakes.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, explanation 1

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what grammar mistakes do you see? Explain them. If there's no mistakes, write 'None'"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, explanation 2

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
      t = line[3]
      prompt = f"I wrote a sentence in English, but I'm not sure it's grammatically correct. Here's my sentence: {t} 1. Are there any strict grammar mistakes in my sentence? Yes or No; 2. If yes, what grammar mistakes do you see? Explain them. If there's no mistakes, write 'None'. Give the answer to each of the points on a separate line"
      prompt_template=f'''SYSTEM: You are an English teacher. Always answer as helpfully, with good explanations of grammar and lexical mistakes.
      USER: {prompt}
      ASSISTANT:
      '''
      response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

      a = response["choices"][0]["text"]
      print(a)
      answers.append(a)

In [ ]:
# label, correction 1

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
      t = line[3]
      prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any strict grammar mistakes in my sentence? Write 'Yes.' or 'No.'; 2. If there is a mistake, what is the correct version of the sentence? Write only the correct version of the sentence and nothing else."
      prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
      USER: {prompt}
      ASSISTANT:
      '''
      response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

      a = response["choices"][0]["text"]
      print(a)
      answers.append(a)

In [ ]:
# label, correction 2

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
      t = line[3]
      prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any strict grammar mistakes in my sentence? Write 'Yes.' or 'No.'; 2. If there is a mistake, what is the correct version of the sentence? Write only the correct version of the sentence and nothing else."
      prompt_template=f'''SYSTEM: You are an English teacher. Always answer as helpfully, with good explanations of grammar and lexical mistakes.
      USER: {prompt}
      ASSISTANT:
      '''
      response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

      a = response["choices"][0]["text"]
      print(a)
      answers.append(a)

In [ ]:
# label, correction 2

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
      t = line[3]
      prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any strict grammar mistakes in my sentence? Write 'Yes.' or 'No.'; 2. If there is a mistake, what is the correct version of the sentence? Write only the correct version of the sentence and nothing else."
      prompt_template=f'''SYSTEM: You are an English teacher. Always answer as helpfully, with good explanations of grammar and lexical mistakes.
      USER: {prompt}
      ASSISTANT:
      '''
      response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

      a = response["choices"][0]["text"]
      print(a)
      answers.append(a)

In [ ]:
# label, category 1

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what is the category of the mistake: syntax, morphology or semantics ? If there's no mistakes, write 'None'"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, category 2

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    # print(y_true)
    prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what is the category of the mistake: syntax, morphology or semantics ? If there's no mistakes, write 'None'"
    prompt_template=f'''SYSTEM: You are an English teacher. Always answer as helpfully, with good explanations of grammar and lexical mistakes.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, category, explanation

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
      t = line[3]
      prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what is the category of the mistake: syntax, morphology or semantics ? If there's no mistakes, write 'None'; 3. Explain the mistakes you see. If there's no mistakes, write 'None'"
      prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
      USER: {prompt}
      ASSISTANT:
      '''
      response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

      a = response["choices"][0]["text"]
      print(a)
      answers.append(a)

In [ ]:
# label, explanation, correction 1

from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  answers = []
  data = []
  for row in df:
      data.append(row)
  for line in data:
    t = line[3]
    prompt = f"Hi! I wrote a sentence in English, but I'm not sure it's grammatically correct. Here's my sentence: {t} 1. Are there any strict grammar mistakes in my sentence? Yes or No. 2. If yes, what grammar mistakes do you see? Explain them. If there's no mistakes, write 'None' 3. What is the correct version of the sentence? Write it like 'sentence', please. If there are no errors in my sentence, just write it again in ' marks like 'sentence'"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, category, correction1 (assistant)
from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what is the category of the mistake: syntax, morphology or semantics ? If there's no mistakes, write 'None'; 3. What is the correct version of the sentence?"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, category, correction2
from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    # print(y_true)
    prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what is the category of the mistake: syntax, morphology or semantics ? If there's no mistakes, write 'None'; 3. What is the correct version of the sentence?"
    prompt_template=f'''SYSTEM: You are an English teacher. Always answer as helpfully, with good explanations of grammar and lexical mistakes.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)

In [ ]:
# label, category, explanation, correction 1
from google.colab import drive
import csv

import numpy
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

drive.mount('/content/drive')

with open('/content/drive/MyDrive/feeding/in_domain_dev.tsv') as f:
  df = csv.reader(f, delimiter="\t")
  data = []
  for row in df:
      data.append(row)
  answers = []
  for line in data:
    t = line[3]
    # print(y_true)
    prompt = f"I wrote a sentence in English, but I'm not sure if it's grammatically correct. Here's my sentence: {t} 1. Are there any grammar mistakes in my sentence? Yes or No; 2. If yes, what is the category of the mistake: syntax, morphology or semantics ? If there's no mistakes, write 'None'; 3. Explain the mistakes you see. If there's no mistakes, write 'None'; 4. What is the correct version of the sentence?"
    prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.
    USER: {prompt}
    ASSISTANT:
    '''
    response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.3, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

    a = response["choices"][0]["text"]
    print(a)
    answers.append(a)